# 谢尔宾斯基分形的绘制

In [ ]:
`pip install numpy matplotlib einops torch

In [ ]:
def get_status_3d(x, y, z, level=3): # 给定一个点，判断是亮的还是暗的，立体
    cur_x = x
    cur_y = y
    cur_z = z
    res_x = 0
    res_y = 0
    res_z = 0
    for i in range(level): # 分形阶数
        res_x = int(cur_x * 3)
        res_y = int(cur_y * 3)
        res_z = int(cur_z * 3)
        cur_x = cur_x * 3 - res_x
        cur_y = cur_y * 3 - res_y
        cur_z = cur_z * 3 - res_z
        if res_y * res_z == 1:
            return False
        if res_x * res_z == 1:
            return False
        if res_x * res_y == 1:
            return False
    return True

fr = 0.1 # 采样率
start = 0
stop = int(1 / fr)
level = 5
for x in range(start, stop):
    for y in range(start, stop - x):
        for z in range(start, stop - x - y):
            xx = x * fr
            yy = y * fr
            zz = z * fr
            if get_status_3d(xx, yy, zz, level=level):
                print(xx,yy,zz)
# 朴素方法

In [ ]:
import numpy as np

fr = 0.001
start = 0
stop = int(1 / fr)
level = 5
x = np.arange(stop)

cur_x = x * fr
res_x = None
res_xy = 1

import matplotlib.pyplot as plt

for i in range(level):
    res_x = (cur_x * 3).astype(np.int32)
    cur_x = cur_x * 3 - res_x

    res_xy = ( np.einsum('x,y->xy', res_x, res_x) - 1 ) * res_xy
    
    res_xy[res_xy != 0] = 1

plt.imshow(res_xy)
plt.show()
# 二维图形的优化算法

In [ ]:
import numpy as np
import einops

fr = 0.001
start = 0
stop = int(1 / fr)
level = 5
x = np.arange(stop)

cur_x = x * fr
res_x = None
res_xyz = 1

import matplotlib.pyplot as plt

for i in range(level):
    res_x = (cur_x * 3).astype(np.int32)
    cur_x = cur_x * 3 - res_x

    res_x = res_x - 1
    res_x[res_x != 0] = 1
    res_xy = np.einsum('x,y,z->xyz', res_x, res_x, 1 - res_x)
    res_xz = einops.rearrange(res_xy, 'x y z->x z y')
    res_yz = einops.rearrange(res_xy, 'x y z->z x y')
    res_xyz = ( np.einsum('x,y,z->xyz', res_x, res_x, res_x) + res_xy + res_xz + res_yz ) * res_xyz

plt.imshow(res_xyz[:,:,0]) # 切片展示
plt.show()
# 三维图形优化

In [ ]:
import torch
import einops

fr = 0.001
start = 0
stop = int(1 / fr)
level = 5
x = torch.arange(stop)
x.to('cuda:0')

cur_x = x * fr
res_x = None
res_xyz = 1

import matplotlib.pyplot as plt

for i in range(level):
    res_x = (cur_x * 3).int()
    cur_x = cur_x * 3 - res_x

    res_x = res_x - 1
    res_x[res_x != 0] = 1
    res_xy = torch.einsum('x,y,z->xyz', res_x, res_x, 1 - res_x)
    res_xz = einops.rearrange(res_xy, 'x y z->x z y')
    res_yz = einops.rearrange(res_xy, 'x y z->z x y')
    res_xyz = ( torch.einsum('x,y,z->xyz', res_x, res_x, res_x) + res_xy + res_xz + res_yz ) * res_xyz

res_xyz.to('cpu')
plt.imshow(res_xyz[:,:,0])
plt.show()
# torch的GPU方案